<a href="https://colab.research.google.com/github/rho-selynn/592-HW3/blob/Roselynn/Copy_of_Copy_of_592_HW3_1142pm_412022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, sys
from google.colab import drive
# This mouunts your google drive to the current runtime
drive.mount('/content/mnt')
# We define a notebook path
nb_path = '/content/notebooks'
# We create a symbolic link from our drive's default "Colab Notebooks" folder to nb_path
os.symlink('/content/mnt/My Drive/Colab Notebooks/RoverTrainingData_Grid', nb_path)
# Insert nb path
sys.path.insert(0, nb_path)

Mounted at /content/mnt


In [2]:
!unzip '/content/mnt/My Drive/Colab Notebooks/RoverTrainingData_Grid/RoverTrainingData_Grid.zip'

Archive:  /content/mnt/My Drive/Colab Notebooks/RoverTrainingData_Grid/RoverTrainingData_Grid.zip
   creating: RoverTrainingData_Grid/IMG/
  inflating: RoverTrainingData_Grid/IMG/robocam_2022_03_23_15_22_10_540.jpg  
  inflating: RoverTrainingData_Grid/IMG/robocam_2022_03_23_15_22_10_597.jpg  
  inflating: RoverTrainingData_Grid/IMG/robocam_2022_03_23_15_22_10_671.jpg  
  inflating: RoverTrainingData_Grid/IMG/robocam_2022_03_23_15_22_10_732.jpg  
  inflating: RoverTrainingData_Grid/IMG/robocam_2022_03_23_15_22_10_801.jpg  
  inflating: RoverTrainingData_Grid/IMG/robocam_2022_03_23_15_22_10_867.jpg  
  inflating: RoverTrainingData_Grid/IMG/robocam_2022_03_23_15_22_10_935.jpg  
  inflating: RoverTrainingData_Grid/IMG/robocam_2022_03_23_15_22_11_001.jpg  
  inflating: RoverTrainingData_Grid/IMG/robocam_2022_03_23_15_22_11_067.jpg  
  inflating: RoverTrainingData_Grid/IMG/robocam_2022_03_23_15_22_11_135.jpg  
  inflating: RoverTrainingData_Grid/IMG/robocam_2022_03_23_15_22_11_201.jpg  
  i

In [3]:
print(len('RoverTraningData_Grid/IMG'))

25


In [10]:
import cv2 
import os 
import glob 
from google.colab.patches import cv2_imshow
import numpy as np
import torch
from torch import int64
#img_dir = "Images"
#img_dir = "/content/mnt/My Drive/Colab Notebooks/RoverTrainingData_Grid/IMG"
img_dir = "RoverTrainingData_Grid/IMG"
data_path = os.path.join(img_dir,'*g') 
files = glob.glob(data_path) 
Images = [] 
for f1 in files: 
    img = cv2.imread(f1, 0) 
    Images.append(img)
Images = np.array(Images)
Images = (np.divide(Images, 255))
Images = Images.astype(float)

In [11]:
print(len(Images))

2297


In [13]:
controlData = np.loadtxt('RoverTrainingData_Grid/robot_log.csv', delimiter = ';', skiprows = 1, usecols = (1,2), dtype = float)
print(np.max(controlData[:,0]))
controlData[:,0] /= 15

15.0


In [22]:
from torch.utils.data import TensorDataset
ImagesTrain = torch.tensor(Images[0:1000])
steerDataTrain = torch.tensor(controlData[0:1000, 0])
trainSteerDataset = TensorDataset(ImagesTrain, steerDataTrain)
ImagesTest = torch.tensor(Images[1001:2296])
steerDataTest = torch.tensor(controlData[1001:2296, 0])
testSteerDataset = TensorDataset(ImagesTest, steerDataTest)
throttleDataTrain = torch.tensor(controlData[0:1000, 1])
trainThrottleDataset = TensorDataset(ImagesTrain, throttleDataTrain)
throttleDataTest = torch.tensor(controlData[1001:2296, 1])
testThrottleDataset = TensorDataset(ImagesTest, throttleDataTest)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


In [23]:
steerTrainDataLoader = torch.utils.data.DataLoader(trainSteerDataset, batch_size=64, shuffle=True)
steerTestDataLoader = torch.utils.data.DataLoader(testSteerDataset, batch_size=64, shuffle=False)
throttleTrainDataLoader = torch.utils.data.DataLoader(trainThrottleDataset, batch_size=64, shuffle=True)
throttleTestDataLoader = torch.utils.data.DataLoader(testThrottleDataset, batch_size=64, shuffle=False)

In [24]:
class LR(torch.nn.Module):
    def __init__(self):
        super(LR, self).__init__()
        self.linear1 = torch.nn.Linear(160*320, 128) # W:784x128 , b:128x1 , parameters = [W,b]
        self.linear2 = torch.nn.Linear(128, 10)
        self.linear3 = torch.nn.Linear(10, 5)
        self.linear4 = torch.nn.Linear(5, 1)
    
    def forward(self, x):
        x = x.view(-1,160*320)
        transformed_x1 = self.linear1(x)
        transformed_x2 = self.linear2(transformed_x1)
        transformed_x3 = self.linear3(transformed_x2)
        transformed_x4 = self.linear4(transformed_x3)
        return transformed_x4

class LRthrottle(torch.nn.Module):
    def __init__(self):
        super(LRthrottle, self).__init__()
        self.linear1 = torch.nn.Linear(160*320, 128) # W:784x128 , b:128x1 , parameters = [W,b]
        self.linear2 = torch.nn.Linear(128, 10)
        self.linear3 = torch.nn.Linear(10, 5)
        self.linear4 = torch.nn.Linear(5, 1)
        self.activation = torch.nn.ReLU()
    
    def forward(self, x):
        x = x.view(-1,160*320)
        transformed_x1 = self.linear1(x)
        transformed_x2 = self.linear2(transformed_x1)
        transformed_x3 = self.linear3(transformed_x2)
        transformed_x4 = self.linear4(transformed_x3)
        #transformed_x4 = self.activation(transformed_x4)
        return transformed_x4

steerNet = LR()
steerOptimizer = torch.optim.Adam(steerNet.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()
criterionThrottle = torch.nn.L1Loss()
throttleNet = LRthrottle()
throttleOptimizer = torch.optim.Adam(throttleNet.parameters(), lr=0.01)

In [25]:
#steering trainer
train_loss_history = []
test_loss_history = []
for epoch in range(20):
    train_loss = 0.0
    test_loss = 0.0
    for i, data in enumerate(steerTrainDataLoader):
        images, labels = data
        images = images.to(torch.float32)
        labels = labels.to(torch.float32)
        steerOptimizer.zero_grad()
        predicted_output = steerNet(images.float())
        predicted_output = predicted_output.to(torch.float32)
        loss = criterion(predicted_output, labels)
        loss.backward()
        steerOptimizer.step()
        train_loss += loss.item()
    for i, data in enumerate(steerTestDataLoader):
        with torch.no_grad():
            images, labels = data
            predicted_output = steerNet(images.float())
            loss = criterion(predicted_output, labels.long())
            test_loss += loss.item()
    train_loss = train_loss/len(steerTrainDataLoader)
    test_loss = test_loss/len(steerTestDataLoader)
    train_loss_history.append(train_loss)
    test_loss_history.append(test_loss)
    print('Epoch %s finished with train loss %s and test loss %s'%(epoch, train_loss, test_loss))

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([40])) that is different to the input size (torch.Size([40, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([15])) that is different to the input size (torch.Size([15, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, ta

Epoch 0 finished with train loss 10303.19379254803 and test loss 3679.4339541480654
Epoch 1 finished with train loss 730.2551595084369 and test loss 298.20447939918154
Epoch 2 finished with train loss 84.20656319707632 and test loss 38.04270517258417
Epoch 3 finished with train loss 15.624103650450706 and test loss 0.9260616103808085
Epoch 4 finished with train loss 2.353583287447691 and test loss 1.0137418565295993
Epoch 5 finished with train loss 0.372272334061563 and test loss 0.2953108520734878
Epoch 6 finished with train loss 0.2691660961136222 and test loss 0.13495357990974471
Epoch 7 finished with train loss 0.1847156584262848 and test loss 0.14282349612386452
Epoch 8 finished with train loss 0.1740036979317665 and test loss 0.12479842343323287
Epoch 9 finished with train loss 0.17108295345678926 and test loss 0.12865733239977134
Epoch 10 finished with train loss 0.17071199882775545 and test loss 0.1275483171588608
Epoch 11 finished with train loss 0.1710855937562883 and test lo

In [26]:
#throttle trainer
train_loss_history = []
test_loss_history = []
for epoch in range(40):
    train_loss = 0.0
    test_loss = 0.0
    for i, data in enumerate(throttleTrainDataLoader):
        images, labels = data
        images = images.to(torch.float32)
        labels = labels.to(torch.float32)
        throttleOptimizer.zero_grad()
        predicted_output = throttleNet(images.float())
        predicted_output = predicted_output.to(torch.float32)
        loss = criterion(predicted_output, labels)
        loss.backward()
        throttleOptimizer.step()
        train_loss += loss.item()
    for i, data in enumerate(throttleTestDataLoader):
        with torch.no_grad():
            images, labels = data
            predicted_output = throttleNet(images.float())
            loss = criterion(predicted_output, labels.long())
            test_loss += loss.item()
    train_loss = train_loss/len(throttleTrainDataLoader)
    test_loss = test_loss/len(throttleTestDataLoader)
    train_loss_history.append(train_loss)
    test_loss_history.append(test_loss)
    print('Epoch %s finished with train loss %s and test loss %s'%(epoch, train_loss, test_loss))

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([40])) that is different to the input size (torch.Size([40, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([15])) that is different to the input size (torch.Size([15, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, ta

Epoch 0 finished with train loss 29793.449406068772 and test loss 8582.652134486607
Epoch 1 finished with train loss 2745.7962510585785 and test loss 963.8492606026786
Epoch 2 finished with train loss 341.6344483792782 and test loss 152.83268083844865
Epoch 3 finished with train loss 41.17274143174291 and test loss 26.868632816133044
Epoch 4 finished with train loss 6.90234007127583 and test loss 4.528484299069359
Epoch 5 finished with train loss 1.6526457741856575 and test loss 1.466008606411162
Epoch 6 finished with train loss 0.5470630116760731 and test loss 0.2878295984119177
Epoch 7 finished with train loss 0.3131755907088518 and test loss 0.31821400281928836
Epoch 8 finished with train loss 0.2625352190807462 and test loss 0.22342341854458764
Epoch 9 finished with train loss 0.2577801709994674 and test loss 0.24085764303093865
Epoch 10 finished with train loss 0.2533128382638097 and test loss 0.2334344493491309
Epoch 11 finished with train loss 0.25203974079340696 and test loss 0

In [27]:
from torch import int64
import cv2 
import os 
import glob 
from google.colab.patches import cv2_imshow
import numpy as np
import torch
#img_dir = "Images"
img_dir = "RoverTrainingData_Grid/IMG"
data_path = os.path.join(img_dir,'*g') 
files = glob.glob(data_path) 
Images = [] 
for f1 in files: 
    img = cv2.imread(f1, 0) 
    Images.append(img)
Images = np.array(Images)
Images = (np.divide(Images, 255))
Images = Images.astype(float)
Images = torch.tensor(Images)

for i in np.linspace(1, 999, 100).astype(int):
  predicted_output_steer = steerNet(Images[i].float())
  predicted_output_throttle = throttleNet(Images.float()[i])
  correct_steer = controlData[i,0]
  correct_throttle = controlData[i,1]
  print ("correct steer: " + str(correct_steer) + " predicted steer: " + str(predicted_output_steer))
  print ("correct throttle: " + str(correct_throttle) + " predicted throttle: " + str(predicted_output_throttle))

correct steer: 0.0 predicted steer: tensor([[-0.0089]], grad_fn=<AddmmBackward0>)
correct throttle: 0.0 predicted throttle: tensor([[0.4059]], grad_fn=<AddmmBackward0>)
correct steer: 0.0 predicted steer: tensor([[-0.1748]], grad_fn=<AddmmBackward0>)
correct throttle: 0.0 predicted throttle: tensor([[0.3646]], grad_fn=<AddmmBackward0>)
correct steer: -0.7830426666666667 predicted steer: tensor([[-0.0533]], grad_fn=<AddmmBackward0>)
correct throttle: 0.0 predicted throttle: tensor([[0.5263]], grad_fn=<AddmmBackward0>)
correct steer: -1.0 predicted steer: tensor([[0.0861]], grad_fn=<AddmmBackward0>)
correct throttle: 0.0 predicted throttle: tensor([[0.4243]], grad_fn=<AddmmBackward0>)
correct steer: 0.6324920666666666 predicted steer: tensor([[-0.0289]], grad_fn=<AddmmBackward0>)
correct throttle: 1.0 predicted throttle: tensor([[0.5708]], grad_fn=<AddmmBackward0>)
correct steer: 0.35304953333333333 predicted steer: tensor([[-0.1101]], grad_fn=<AddmmBackward0>)
correct throttle: 1.0 pred